In [1]:
import CmtRpyLgcGPTProcess
import os

import yaml
import pandas as pd

# base_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
base_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

event_path = os.path.join(base_dir, 'keyEvent.yaml')
# Load the YAML file
with open(event_path, 'r', encoding='utf-8' ) as file:
    event_config = yaml.safe_load(file)
    print("Event YAML content loaded successfully.")
print(event_config)

Current working directory: /Users/jackhan/Desktop/MultiAgent-Framwork/MultiAgent-Framwork/CmtRpyCtrl
Config sections found: ['mysql', 'Paths', 'OpenAI', 'NetworkSocket']
YAML content loaded successfully.
Event YAML content loaded successfully.
{'npcEvents': [{'npcId': 10016, 'npcName': 'Yves', 'events': [{'id': 'leaving_hollywood', 'intro': 'Left my acting career behind to become a painter.', 'details': ['My old phone was filled with unread messages from producers', 'The sketchboard in my trailer had been untouched for years', 'The folding easel was still packed from my art school days', 'My palette was dry, just like my creative spirit']}, {'id': 'painting_nova', 'intro': 'Found my true calling while painting in the lilac field.', 'details': ['The sketchboard framed her like a perfect shot', 'My palette found colors no camera could capture', "The easel became my director's chair, but this scene was real", 'My phone stayed silent, and I was finally the artist I wanted to be']}]}, {'npc

In [8]:
npcId = 10016

npc_events_entry = next((item for item in event_config.get('npcEvents', []) if item.get('npcId') == npcId), None)
print(npc_events_entry)
if npc_events_entry:
    events_list = npc_events_entry.get('events', [])
    events_data = []
    for ev in events_list:
        ev_id = ev.get('id')
        ev_intro = ev.get('intro')
        ev_detail = str(ev.get('details'))
        ev_embedding = CmtRpyLgcGPTProcess.get_embedding(ev_intro)
        events_data.append([ev_id, ev_intro, ev_detail, ev_embedding])
    events_df = pd.DataFrame(events_data, columns=['eventId', 'intro', 'detail', 'embedding'])
else:
    events_df = pd.DataFrame(columns=['eventId', 'intro', 'detail', 'embedding'])

{'npcId': 10016, 'npcName': 'Yves', 'events': [{'id': 'leaving_hollywood', 'intro': 'Left my acting career behind to become a painter.', 'details': ['My old phone was filled with unread messages from producers', 'The sketchboard in my trailer had been untouched for years', 'The folding easel was still packed from my art school days', 'My palette was dry, just like my creative spirit']}, {'id': 'painting_nova', 'intro': 'Found my true calling while painting in the lilac field.', 'details': ['The sketchboard framed her like a perfect shot', 'My palette found colors no camera could capture', "The easel became my director's chair, but this scene was real", 'My phone stayed silent, and I was finally the artist I wanted to be']}]}


In [9]:
events_df

,eventId,intro,detail,embedding
0,leaving_hollywood,Left my acting career behind to become a painter.,['My old phone was filled with unread messages...,"[0.0006441478035412729, -0.008325561881065369,..."
1,painting_nova,Found my true calling while painting in the li...,['The sketchboard framed her like a perfect sh...,"[-0.015384423546493053, -0.026086006313562393,..."


In [18]:
commet_to_reply = "Tell me about the your career"
comment_str = str(commet_to_reply)
comment_embedding = CmtRpyLgcGPTProcess.get_embedding(comment_str)

In [19]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot_product / (norm1 * norm2)

# compute similarity between the comment and each event
events_df['similary'] = events_df['embedding'].apply(lambda emb: cosine_similarity(emb, comment_embedding))
selected_event = events_df.sort_values(by='similary', ascending=False).head(1)

In [22]:
# Filter for similarity > 0.3 and pick at most top 3 events
threshold = 0.3
selected_event = (
    events_df[events_df['similary'] > threshold]
    .sort_values(by='similary', ascending=False)
    .head(3)
)
selected_event


,eventId,intro,detail,embedding,similary
0,leaving_hollywood,Left my acting career behind to become a painter.,['My old phone was filled with unread messages...,"[0.0006441478035412729, -0.008325561881065369,...",0.368861


In [24]:
import ast

# Build paragraphs by combining intro and detailed points
paragraphs = []
for _, row in selected_event.iterrows():
    # Parse the stringified list of details back into a Python list
    details_list = ast.literal_eval(row['detail'])
    # Join all detail items into a single string
    details_text = " ".join(details_list)
    # Form the paragraph: intro followed by details
    paragraph = f"{row['intro']} {details_text}"
    paragraphs.append(paragraph)


relevent_event = "\n\n".join(
    f"relevent event {idx}: {para}"
    for idx, para in enumerate(paragraphs, start=1)
)


In [25]:
relevent_event

'relevent event 1: Left my acting career behind to become a painter. My old phone was filled with unread messages from producers The sketchboard in my trailer had been untouched for years The folding easel was still packed from my art school days My palette was dry, just like my creative spirit'